In [1]:
import os
import sys

from pyspark.sql.functions import count, col


In [13]:
import pandas as pd
from IPython.display import display

# Adjust Pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows (optional for large datasets)
pd.set_option('display.width', None)        # Expand display width to fit content


In [3]:

import os

# Add the project root directory (CrashAnalysisProject) to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed
sys.path.append(project_root)

# Verify that the src folder is now accessible
print(sys.path)

['/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/prakhar/Desktop/crash_analysis/venv/lib/python3.8/site-packages', '/home/prakhar/Desktop/crash_analysis']


In [4]:
from src.utils.df_processor import DataFrameProcessor

In [5]:
# creating a spark session 
from src.utils.spark_helper import SparkHelper

spark_helper = SparkHelper()

spark = spark_helper.create_spark_session()

print(f"Spark Version: {spark.version}")


25/01/19 23:07:42 WARN Utils: Your hostname, prakhar-Predator-PH315-51 resolves to a loopback address: 127.0.1.1; using 192.168.1.15 instead (on interface wlp0s20f3)
25/01/19 23:07:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/19 23:07:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/19 23:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Version: 3.5.4


### Analyzing Units use

In [6]:
df = spark.read.csv('../data/raw/Units_use.csv', header=True)

In [15]:
df.show(5)

+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+------------+----------+--------------------+---------------+--------+-----------------+--------------------+--------------------+-----------------+--------------+--------------------+-----------------+--------------+------------------+---------------+--------------------+--------------------+------------------+-------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR| UNIT_DESC_ID|VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|              VIN|VEH_MOD_YEAR|VEH_COLOR_ID| VEH_MAKE_ID|VEH_MOD_ID|    VEH_BODY_STYL_ID|EMER_RESPNDR_FL|OWNR_ZIP|FIN_RESP_PROOF_ID|    FIN_RESP_TYPE_ID|  VEH_DMAG_AREA_1_ID|VEH_DMAG_SCL_1_ID|FORCE_DIR_1_ID|  VEH_DMAG_AREA_2_ID|VEH_DMAG_SCL_2_ID|FORCE_DIR_2_ID|VEH_INVENTORIED_FL|VEH_TRANSP_NAME|     VEH_TRANSP_DEST|  CONTRIB_FACT

In [9]:
df_pandas = df.select(col('VEH_BODY_STYL_ID')).distinct().toPandas()

In [14]:
display(df_pandas)

,VEH_BODY_STYL_ID
0,BUS
1,NA
2,VAN
3,PICKUP
4,SPORT UTILITY VEHICLE
5,"PASSENGER CAR, 4-DOOR"
6,FIRE TRUCK
7,TRUCK
8,UNKNOWN
9,AMBULANCE


In [16]:
# filtering for 2-wheelers
df_filtered = df.filter((col('VEH_BODY_STYL_ID') == 'MOTORCYCLE') | (col('VEH_BODY_STYL_ID') == 'POLICE MOTORCYCLE'))

In [19]:
# count of records 
df_filtered.count()

784

In [20]:
# count of distinct number plates of 2 wheerers
df_filtered.select('VIN').distinct().count()

767

In [27]:
# checking if there are multiple record for same vehicle 
df_filtered.groupBy('VIN').agg(count("*").alias('cnt')).select('VIN', 'cnt').orderBy(col('cnt').desc()).show()

+-----------------+---+
|              VIN|cnt|
+-----------------+---+
|             NULL|  4|
|1V9FV31A44R116744|  2|
|JH2PC2102VM800659|  2|
|1HD4CAM12XK138007|  2|
|L5YTCKPA691107388|  2|
|L1ETCBMJXCA000101|  2|
|4K7S81358YC009326|  2|
|4B7H84691TS004414|  2|
|1A9HSWHD26N428188|  2|
|5KMMSG2PX67001970|  2|
|5VPDW36N8E3034778|  2|
|1N9HW21035N209018|  2|
|JYAVP24E97A002526|  2|
|LKDTCLTG3F1C10094|  2|
|LD6PCK0B9FL100221|  2|
|1HD1TAN14FB957441|  1|
|JH2PC40078M101884|  1|
|4MZKP01D193000375|  1|
|1HD1BMY191Y072190|  1|
|1HD1KBM18BB679048|  1|
+-----------------+---+
only showing top 20 rows



In [29]:
df_temp = df_filtered.filter(col('VIN')=='1V9FV31A44R116744').toPandas()

In [30]:
display(df_temp)

,CRASH_ID,UNIT_NBR,UNIT_DESC_ID,VEH_PARKED_FL,VEH_HNR_FL,VEH_LIC_STATE_ID,VIN,VEH_MOD_YEAR,VEH_COLOR_ID,VEH_MAKE_ID,VEH_MOD_ID,VEH_BODY_STYL_ID,EMER_RESPNDR_FL,OWNR_ZIP,FIN_RESP_PROOF_ID,FIN_RESP_TYPE_ID,VEH_DMAG_AREA_1_ID,VEH_DMAG_SCL_1_ID,FORCE_DIR_1_ID,VEH_DMAG_AREA_2_ID,VEH_DMAG_SCL_2_ID,FORCE_DIR_2_ID,VEH_INVENTORIED_FL,VEH_TRANSP_NAME,VEH_TRANSP_DEST,CONTRIB_FACTR_1_ID,CONTRIB_FACTR_2_ID,CONTRIB_FACTR_P1_ID,VEH_TRVL_DIR_ID,FIRST_HARM_EVT_INV_ID,INCAP_INJRY_CNT,NONINCAP_INJRY_CNT,POSS_INJRY_CNT,NON_INJRY_CNT,UNKN_INJRY_CNT,TOT_INJRY_CNT,DEATH_CNT
0,14926725,1,MOTOR VEHICLE,N,N,TX,1V9FV31A44R116744,2004,BLK,UNKNOWN,UNKNOWN,MOTORCYCLE,N,77573,1,PROOF OF LIABILITY INSURANCE,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,12,NA,NA,NA,Y,"BOSONE, INC","BOSONE, INC",NONE,NA,NA,EAST,Y,0,1,0,0,0,1,0
1,14926725,1,MOTOR VEHICLE,N,N,TX,1V9FV31A44R116744,2004,BLK,UNKNOWN,UNKNOWN,MOTORCYCLE,N,77573,1,PROOF OF LIABILITY INSURANCE,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,12,NA,NA,NA,Y,"BOSONE, INC","BOSONE, INC",NONE,NA,NA,EAST,Y,0,1,0,0,0,1,0


In [31]:
df_temp = df_filtered.filter(col('VIN')=='JH2PC2102VM800659').toPandas()

In [32]:
display(df_temp)

,CRASH_ID,UNIT_NBR,UNIT_DESC_ID,VEH_PARKED_FL,VEH_HNR_FL,VEH_LIC_STATE_ID,VIN,VEH_MOD_YEAR,VEH_COLOR_ID,VEH_MAKE_ID,VEH_MOD_ID,VEH_BODY_STYL_ID,EMER_RESPNDR_FL,OWNR_ZIP,FIN_RESP_PROOF_ID,FIN_RESP_TYPE_ID,VEH_DMAG_AREA_1_ID,VEH_DMAG_SCL_1_ID,FORCE_DIR_1_ID,VEH_DMAG_AREA_2_ID,VEH_DMAG_SCL_2_ID,FORCE_DIR_2_ID,VEH_INVENTORIED_FL,VEH_TRANSP_NAME,VEH_TRANSP_DEST,CONTRIB_FACTR_1_ID,CONTRIB_FACTR_2_ID,CONTRIB_FACTR_P1_ID,VEH_TRVL_DIR_ID,FIRST_HARM_EVT_INV_ID,INCAP_INJRY_CNT,NONINCAP_INJRY_CNT,POSS_INJRY_CNT,NON_INJRY_CNT,UNKN_INJRY_CNT,TOT_INJRY_CNT,DEATH_CNT
0,14936734,2,MOTOR VEHICLE,N,Y,TX,JH2PC2102VM800659,1997,BLK,HONDA,UNKNOWN,MOTORCYCLE,N,79763,1,PROOF OF LIABILITY INSURANCE,BACK END DAMAGE DISTRIBUTED IMPACT,DAMAGED 1 MINIMUM,6,NA,NA,NA,None,None,None,CHANGED LANE WHEN UNSAFE,NA,NA,WEST,Y,0,0,0,1,0,0,0
1,15258469,2,MOTOR VEHICLE,N,N,TX,JH2PC2102VM800659,1997,BLK,HONDA,UNKNOWN,MOTORCYCLE,N,79763,1,PROOF OF LIABILITY INSURANCE,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,6,NA,NA,NA,N,None,None,NONE,NA,NA,WEST,Y,0,0,0,1,0,0,0


In [35]:
df_temp = df_filtered.filter(col('VIN').isNull()).toPandas()

In [36]:
display(df_temp)

,CRASH_ID,UNIT_NBR,UNIT_DESC_ID,VEH_PARKED_FL,VEH_HNR_FL,VEH_LIC_STATE_ID,VIN,VEH_MOD_YEAR,VEH_COLOR_ID,VEH_MAKE_ID,VEH_MOD_ID,VEH_BODY_STYL_ID,EMER_RESPNDR_FL,OWNR_ZIP,FIN_RESP_PROOF_ID,FIN_RESP_TYPE_ID,VEH_DMAG_AREA_1_ID,VEH_DMAG_SCL_1_ID,FORCE_DIR_1_ID,VEH_DMAG_AREA_2_ID,VEH_DMAG_SCL_2_ID,FORCE_DIR_2_ID,VEH_INVENTORIED_FL,VEH_TRANSP_NAME,VEH_TRANSP_DEST,CONTRIB_FACTR_1_ID,CONTRIB_FACTR_2_ID,CONTRIB_FACTR_P1_ID,VEH_TRVL_DIR_ID,FIRST_HARM_EVT_INV_ID,INCAP_INJRY_CNT,NONINCAP_INJRY_CNT,POSS_INJRY_CNT,NON_INJRY_CNT,UNKN_INJRY_CNT,TOT_INJRY_CNT,DEATH_CNT
0,14898984,2,MOTOR VEHICLE,N,N,UN,None,NA,BLK,KAWASAKI,NA,MOTORCYCLE,N,None,2,NA,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,12,NA,NA,NA,Y,MISSION TOWING,GROWDON POUND,DRIVER INATTENTION,FOLLOWED TOO CLOSELY,UNSAFE SPEED,NORTH,N,0,1,0,0,0,1,0
1,15110581,1,MOTOR VEHICLE,N,N,TX,None,2000,GRN,HARLEY-DAVIDSON,UNKNOWN,MOTORCYCLE,N,76513,1,LIABILITY INSURANCE POLICY,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,NA,NA,NA,NA,N,LOOP 121 TOWING,LOOP 121 TOWING,FAILED TO YIELD ROW - EMERGENCY VEHICLE,OTHER (EXPLAIN IN NARRATIVE),NA,EAST,Y,0,1,0,0,0,1,0
2,15175697,1,MOTOR VEHICLE,N,N,TX,None,NA,BLK,HONDA,NA,MOTORCYCLE,N,77082,2,NA,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,NA,NA,NA,NA,Y,None,None,NONE,NA,NA,WEST,Y,1,0,0,0,0,1,0
3,15227142,1,MOTOR VEHICLE,N,N,TX,None,NA,WHI,YAMAHA,NA,MOTORCYCLE,N,78418,2,NA,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,3,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,3,N,APOLLO TOWING COMPANY,6342 HARWICK STREET CORPUS CHRISTI TX 78,OTHER (EXPLAIN IN NARRATIVE),NA,UNSAFE SPEED,WEST,Y,1,0,0,0,0,1,0


In [37]:
# checking owner zip of 2 wheelers 
df_filtered.groupBy('OWNR_ZIP').agg(count("*").alias('cnt')).select('OWNR_ZIP', 'cnt').orderBy(col('cnt').desc()).show()

+--------+---+
|OWNR_ZIP|cnt|
+--------+---+
|    NULL| 14|
|   79924|  7|
|   78758|  6|
|   78660|  6|
|   79605|  6|
|   79936|  6|
|   79938|  5|
|   78745|  5|
|   77511|  5|
|   78741|  5|
|   77840|  5|
|   79912|  5|
|   77316|  4|
|   75067|  4|
|   78108|  4|
|   78541|  4|
|   78155|  4|
|   78613|  4|
|   76179|  4|
|   78521|  4|
+--------+---+
only showing top 20 rows



In [39]:
df_temp = df_filtered.filter(col('OWNR_ZIP')=='76513').toPandas()
display(df_temp)

,CRASH_ID,UNIT_NBR,UNIT_DESC_ID,VEH_PARKED_FL,VEH_HNR_FL,VEH_LIC_STATE_ID,VIN,VEH_MOD_YEAR,VEH_COLOR_ID,VEH_MAKE_ID,VEH_MOD_ID,VEH_BODY_STYL_ID,EMER_RESPNDR_FL,OWNR_ZIP,FIN_RESP_PROOF_ID,FIN_RESP_TYPE_ID,VEH_DMAG_AREA_1_ID,VEH_DMAG_SCL_1_ID,FORCE_DIR_1_ID,VEH_DMAG_AREA_2_ID,VEH_DMAG_SCL_2_ID,FORCE_DIR_2_ID,VEH_INVENTORIED_FL,VEH_TRANSP_NAME,VEH_TRANSP_DEST,CONTRIB_FACTR_1_ID,CONTRIB_FACTR_2_ID,CONTRIB_FACTR_P1_ID,VEH_TRVL_DIR_ID,FIRST_HARM_EVT_INV_ID,INCAP_INJRY_CNT,NONINCAP_INJRY_CNT,POSS_INJRY_CNT,NON_INJRY_CNT,UNKN_INJRY_CNT,TOT_INJRY_CNT,DEATH_CNT
0,15110581,1,MOTOR VEHICLE,N,N,TX,None,2000,GRN,HARLEY-DAVIDSON,UNKNOWN,MOTORCYCLE,N,76513,1,LIABILITY INSURANCE POLICY,"MOTORCYCLE, SCOOTER, MOPED, ETC. ONLY",DAMAGED 1 MINIMUM,NA,NA,NA,NA,N,LOOP 121 TOWING,LOOP 121 TOWING,FAILED TO YIELD ROW - EMERGENCY VEHICLE,OTHER (EXPLAIN IN NARRATIVE),NA,EAST,Y,0,1,0,0,0,1,0
1,15107598,2,MOTOR VEHICLE,N,N,TX,JS1VS52A642100742,2004,BLK,SUZUKI,NA,MOTORCYCLE,N,76513,1,PROOF OF LIABILITY INSURANCE,FRONT END DAMAGE CONCENTRATED IMPACT,DAMAGED 5,12,NA,NA,NA,N,BIG JOHNS,"1502 HOLLAND RD, BELTON, TX 76513",NONE,NA,NA,WEST,Y,0,0,0,0,0,0,1
2,15388050,1,MOTOR VEHICLE,N,N,TX,1RD1GX4117K301434,2007,BLK,HARLEY-DAVIDSON,NA,MOTORCYCLE,N,76513,2,NA,RIGHT SIDE DAMAGE ANGULAR IMPACT,DAMAGED 5,3,NA,NA,NA,N,None,None,UNDER INFLUENCE - ALCOHOL,FAILED TO CONTROL SPEED,NA,EAST,Y,1,0,0,0,0,1,0


In [49]:
df_filtered = df.filter((col('VEH_BODY_STYL_ID') == 'MOTORCYCLE') | (col('VEH_BODY_STYL_ID') == 'POLICE MOTORCYCLE'))
df_filtered = df_filtered.filter(col("VIN").isNotNull())

# Remove duplicates and count distinct records
distinct_count = df_filtered.select("VIN").distinct().count()


In [50]:
distinct_count

766

In [46]:
df.count()

173499

In [ ]:
df.